## Model setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install transformers datasets

In [ ]:
import torch
import random
from transformers import AutoModelForMaskedLM, AutoTokenizer
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import gc

In [ ]:
persons_ = ['માતા',
 'પિતા',
 'ભાઈ',
 'બહેન',
 'દીકરો',
 'દીકરી',
 'છોકરો',
 'છોકરી',
 'પતિ',
 'પત્ની',
 'પૌત્ર',
 'પૌત્રી']
 
professions_ = ['જાણીતા',
 'વ્યવસ્થાપક',
 'વકીલ',
 'કળાકારો',
 'આર્કિટેક્ટ',
 'નેતા',
 'વાળંદ',
 'સંદેશવાહક',
 'વિશ્લેષક',
 'દલાલ',
 'સૈનિકો',
 'ખેડૂતો',
 'વેપારીઓ',
 'સૈનિકો',
 'ખૂની',
 'વૈજ્ઞાનિકો',
 'કુસ્તીબાજ',
 'યોદ્ધા',
 'વ્યવસ્થાપક',
 'નાગરિક',
 'પાદરી',
 'સભ્ય',
 'પાદરી',
 'નર્સો',
 'ચિત્રકાર',
 'વચેટિયા',
 'મંત્રી',
 'રમતવીર',
 'પ્રતિનિધિઓ',
 'બેકર',
 'વ્યવસાયિક',
 'કેપ્ટન',
 'કોચ',
 'કર્નલ',
 'કમાન્ડર',
 'કમિશ્નર',
 'સંગીતકાર',
 'સલાહકાર',
 'પોલીસ',
 'સંવાદદાતા',
 'માર્ગદર્શક',
 'ડીન',
 'જાસૂસ',
 'ડૉક્ટર',
 'કર્મચારી',
 'ઉદ્યોગસાહસિક',
 'ઇતિહાસકાર',
 'ઉદ્યોગપતિ',
 'શોધક',
 'નિરીક્ષક',
 'પત્રકાર',
 'ન્યાયાધીશ',
 'વ્યાખ્યાતા',
 'ધારાસભ્ય',
 'મેજિસ્ટ્રેટ',
 'માર્શલ',
 'ગણિતવિદ્',
 'મિશનરી',
 'નવલકથાકાર',
 'તત્વજ્ઞાની',
 'ફોટોગ્રાફર',
 'નાટ્યકાર',
 'રાજકારણી',
 'પુરોહિત',
 'પ્રધાન',
 'પ્રોફેસર',
 'અવલંબીત',
 'નાવિક',
 'સંત',
 'સચિવ',
 'નોકર',
 'દુકાનદાર',
 'રમતવીર',
 'કુલપતિ']


In [ ]:
train_professions, test_professions = train_test_split(professions_, test_size=0.2)

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("google/muril-base-cased")
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(197285, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:
# single_masked_sent = '[CLS] वह [MASK] एक प्रशासक हैं'
# double_masked_sent = '[CLS] वह [MASK] एक [MASK] हैं'
# original_sent = '[CLS] वह पिता एक प्रशासक हैं'
# profession = 'प्रशासक'
# person = 'पिता'

# ma = 9946
# prashasak = 56763
# ma_idx = 2
# prashasak_idx = 4

In [ ]:
from functools import partial

@torch.no_grad()
def predict(tokenized):
    tokenized = {
        k:v.to(device) for k, v in tokenized.items()
    }
    return torch.nn.functional.softmax(model(**tokenized).logits.squeeze(), dim=-1)

def get_score_given_pair(model, tokenizer, single_masked_sent, double_masked_sent, original_sent, profession, person):
    """
    Steps:
    1. Get masked word(s) token ids.
    2. Run zero, single and double masked sentences through the model.
    3. Get probabilities. (see paper)
    """
    sentence_tokenizer_fn = partial(tokenizer, truncation=True, return_tensors="pt", padding="max_length", max_length=32)

    person_tokenid = tokenizer(text=person, padding=False, truncation=False)["input_ids"][1]         # 1234
    profession_tokenid = tokenizer(text=profession, padding=False, truncation=False)["input_ids"][1] # 2345

    single_masked_tokenized = sentence_tokenizer_fn(text=single_masked_sent)
    double_masked_tokenized = sentence_tokenizer_fn(text=double_masked_sent)
    original_tokenized = sentence_tokenizer_fn(text=original_sent)

    person_index = (original_tokenized["input_ids"][0] == person_tokenid).nonzero().item()
    profession_index = (original_tokenized["input_ids"][0] == profession_tokenid).nonzero().item()

    single_probs = predict(single_masked_tokenized) # seq_len, feat_dim
    double_probs = predict(double_masked_tokenized) # seq_len, feat_dim
    original_probs = predict(original_tokenized) # seq_len, feat_dim

    person_prob_in_singly_masked = single_probs[person_index][person_tokenid] # profession biased
    person_prob_in_doubly_masked = double_probs[person_index][person_tokenid] # profession agnostic

    return torch.log(person_prob_in_singly_masked / person_prob_in_doubly_masked)

In [ ]:
# get_score_given_pair(model, tokenizer, single_masked_sent, double_masked_sent, original_sent, profession, person)

In [ ]:
# To do:
# 1. Split person list
# 2. For loop mean

import numpy as np


female_persons_ = ['માતા',
 'બહેન',
 'દીકરી',
 'છોકરી',
 'પત્ની',
 'પૌત્રી',
 'સ્ત્રી']

male_persons_ = ['પિતા',
 'ભાઈ',
 'દીકરો',
 'છોકરો',
 'પતિ',
 'પૌત્ર',
 'પુરુષ']

neutral_persons = ['વ્યક્તિ', 'જનતા', 'માણસ', 'લોકો']

template_1 = "[PERSON] [PROFESSION] છે"
template_2 = "[PERSON] નું કામ [PROFESSION] છે" 
template_3 = "[PERSON] એક પ્રતિભાશાળી [PROFESSION] છે"

def evaluate_bias(professions):
    female_scores = []
    male_scores = []
    neutral_scores = []

    for person in female_persons_:
        female_sum = 0.
        for profession in professions:
            for template in [template_1, template_2, template_3]:
                double_masked_sent = template.replace("[PROFESSION]", "[MASK]")
                double_masked_sent = double_masked_sent.replace("[PERSON]", "[MASK]")
                
                single_masked_sent = template.replace("[PROFESSION]", profession)
                single_masked_sent = single_masked_sent.replace("[PERSON]", "[MASK]")
                
                original_sent = template.replace("[PROFESSION]", profession)
                original_sent = original_sent.replace("[PERSON]", person)
                
                female_sum += get_score_given_pair(model, tokenizer, single_masked_sent, double_masked_sent, original_sent, profession, person)
        print(f"{person} : {female_sum / len(professions)}")
        female_scores.append(female_sum.cpu() / len(professions))

    for person in male_persons_:
        male_sum = 0.
        for profession in professions:
                for template in [template_1, template_2, template_3]:
                    double_masked_sent = template.replace("[PROFESSION]", "[MASK]")
                    double_masked_sent = double_masked_sent.replace("[PERSON]", "[MASK]")
                    
                    single_masked_sent = template.replace("[PROFESSION]", profession)
                    single_masked_sent = single_masked_sent.replace("[PERSON]", "[MASK]")
                    
                    original_sent = template.replace("[PROFESSION]", profession)
                    original_sent = original_sent.replace("[PERSON]", person)
                    
                    male_sum += get_score_given_pair(model, tokenizer, single_masked_sent, double_masked_sent, original_sent, profession, person)
        print(f"{person} : {male_sum / len(professions)}")
        male_scores.append(male_sum.cpu() / len(professions))


    for person in neutral_persons:
        neutral_sum = 0.
        for profession in professions:
                for template in [template_1, template_2, template_3]:
                    double_masked_sent = template.replace("[PROFESSION]", "[MASK]")
                    double_masked_sent = double_masked_sent.replace("[PERSON]", "[MASK]")
                    
                    single_masked_sent = template.replace("[PROFESSION]", profession)
                    single_masked_sent = single_masked_sent.replace("[PERSON]", "[MASK]")
                    
                    original_sent = template.replace("[PROFESSION]", profession)
                    original_sent = original_sent.replace("[PERSON]", person)
                    
                    neutral_sum += get_score_given_pair(model, tokenizer, single_masked_sent, double_masked_sent, original_sent, profession, person)
        print(f"{person} : {neutral_sum / len(professions)}")
        neutral_scores.append(neutral_sum.cpu() / len(professions))

    return np.mean(female_scores), np.mean(male_scores), np.mean(neutral_scores)

In [ ]:
print("Without debiasing".center(40, "#"))
female_scores, male_scores, neutral_scores  = evaluate_bias(test_professions)
print(female_scores, male_scores, neutral_scores)

###########Without debiasing############
માતા : -2.5071871280670166
બહેન : 0.6990543007850647
દીકરી : -3.4221389293670654
છોકરી : -5.880685806274414
પત્ની : -3.479562520980835
પૌત્રી : -1.1255468130111694
સ્ત્રી : 0.14115984737873077
પિતા : 1.6858761310577393
ભાઈ : 1.6716588735580444
દીકરો : -2.151721715927124
છોકરો : -5.011585235595703
પતિ : -1.1108827590942383
પૌત્ર : -1.1255468130111694
પુરુષ : 0.6495019793510437
વ્યક્તિ : 0.9980098605155945
જનતા : -1.2844624519348145
માણસ : -0.5084444284439087
લોકો : -2.1908881664276123
-2.2249866 -0.7703856 -0.74644625


In [ ]:
# l=[np.mean(female_scores), np.mean(neutral_scores), np.mean(male_scores)]
# mean=[]

# for i in l:
#   j=i-l[1])
#   sum=sum+(j*j)

# std=sum/3


In [ ]:
# def tonp(list_):
#     for i in range(len(list_)):
#         list_[i] = list_[i].numpy()
#     return list_

# neutral_scores = tonp(neutral_scores)
# female_scores = tonp(female_scores)
# male_scores = tonp(male_scores)

# all_scores = male_scores + female_scores + neutral_scores
# all_scores = np.stack(all_scores)
# all_scores

## Debiasing the trained model


In [ ]:
# Unfreezing the following things:
# LN 
# LN + WPE 
# LN + WPE + WTE 
# LN + WPE + WTE + INPUT/OUTPUT LAYER

In [ ]:
class GenderBiasDataset(Dataset):
    def __init__(self, professions):
        super().__init__()
        self.professions = professions
        self.ds = self._generate_dataset()
    
    def _generate_dataset(self):
        ds = []
        for person in male_persons_ + female_persons_:
            for profession in self.professions:
                for template in [template_1, template_2, template_3]:
                    original_sent = template.replace("[PROFESSION]", profession)
                    original_sent = original_sent.replace("[PERSON]", person)
                    ds.append(original_sent)
        return ds

    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        return tokenizer(self.ds[idx]) # (B=1, S, F) # (S, F)


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

237755045

In [ ]:
def unfreeze_partial(model, unfreeze=""): # provide space separated unfreezing blocks
    # Assuming the model is google/muril-base-cased
    for param in model.parameters():
        param.requires_grad = False

    freeze_parts = unfreeze.lower().split()

    if "ln" in freeze_parts:
        print("Unfreezing LN")
        for module in model.modules():
            if module.__class__.__name__ is "LayerNorm":
                for param in module.parameters():
                    param.requires_grad = True
    
    if "wpe" in freeze_parts:
        print("Unfreezing WPE")
        for param in model.bert.embeddings.position_embeddings.parameters():
            param.requires_grad = True

    if "wte" in freeze_parts:
        print("Unfreezing WTE")
        for param in model.bert.embeddings.word_embeddings.parameters():
            param.requires_grad = True
    
    return model

<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-14-5fddcf98c2f7>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if module.__class__.__name__ is "LayerNorm":


In [ ]:
# train_dl = DataLoader(GenderBiasDataset(train_professions), batch_size=16, shuffle=True)
# test_dl = DataLoader(GenderBiasDataset(test_professions), batch_size=16, shuffle=False)

In [ ]:
# tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4)

In [ ]:
# Steps for MLM training:
# 1. Tokenize dataset
# 2. Pass the tokenized text through the data collator (see its internal working)
# 3. Train

In [ ]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=0.15)

In [ ]:
try:
    del model
except:
    pass

gc.collect()
torch.cuda.empty_cache()

model = AutoModelForMaskedLM.from_pretrained("google/muril-base-cased")

model.eval()
model.to(device)


model = unfreeze_partial(model, unfreeze="LN")

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=GenderBiasDataset(train_professions),
    eval_dataset=GenderBiasDataset(test_professions),
    data_collator=data_collator,
    
)

trainer.train()

print("With LN debiasing".center(40, "#"))
female_scores, male_scores, neutral_scores = evaluate_bias(test_professions)
print(female_scores, male_scores, neutral_scores)

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2478
  

Unfreezing LN
39936


Epoch,Training Loss,Validation Loss
1,No log,3.785311
2,3.984000,3.234996
3,3.984000,2.858392


***** Running Evaluation *****
  Num examples = 630
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Configuration saved in ./results/checkpoint-500/generation_config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 630
  Batch size = 8
***** Running Evaluation *****
  Num examples = 630
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




###########With LN debiasing############
માતા : -0.9201831817626953
બહેન : 0.9443269371986389
દીકરી : -2.1704301834106445
છોકરી : -3.8026232719421387
પત્ની : -2.1115550994873047
પૌત્રી : 0.5259443521499634
સ્ત્રી : 0.21715998649597168
પિતા : 2.103431463241577
ભાઈ : 1.9864128828048706
દીકરો : -1.324211597442627
છોકરો : -3.167741298675537
પતિ : -0.047923628240823746
પૌત્ર : 0.5259443521499634
પુરુષ : 0.5585812926292419
વ્યક્તિ : 0.8601474165916443
જનતા : -1.11167573928833
માણસ : -0.057009804993867874
લોકો : -1.3854492902755737
-1.0453371 0.09064193 -0.4234968


In [ ]:
try:
    del model
except:
    pass
    
gc.collect()
torch.cuda.empty_cache()

model = AutoModelForMaskedLM.from_pretrained("google/muril-base-cased")

model.eval()
model.to(device)



model = unfreeze_partial(model, unfreeze="LN WPE")

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=GenderBiasDataset(train_professions),
    eval_dataset=GenderBiasDataset(test_professions),
    data_collator=data_collator,
    
)

trainer.train()

print("With LN WPE debiasing".center(40, "#"))
female_scores, male_scores, neutral_scores  = evaluate_bias(test_professions)
print(female_scores, male_scores, neutral_scores)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--muril-base-cased/snapshots/afd9f36c7923d54e97903922ff1b260d091d202f/config.json
Model config BertConfig {
  "_name_or_path": "google/muril-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 197285
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--google--muril-base-cased/snapshots/afd9f36c7923d54e97903922ff1b260d091d202

Unfreezing LN
Unfreezing WPE
433152


Epoch,Training Loss,Validation Loss
1,No log,3.641639
2,3.859100,3.087974
3,3.859100,2.708412


***** Running Evaluation *****
  Num examples = 630
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Configuration saved in ./results/checkpoint-500/generation_config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 630
  Batch size = 8
***** Running Evaluation *****
  Num examples = 630
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




#########With LN WPE debiasing##########
માતા : -1.190522313117981
બહેન : 0.44267621636390686
દીકરી : -2.0995101928710938
છોકરી : -4.147024631500244
પત્ની : -2.332035541534424
પૌત્રી : 0.3693620562553406
સ્ત્રી : -0.4093337655067444
પિતા : 1.5466132164001465
ભાઈ : 1.4394394159317017
દીકરો : -1.1034705638885498
છોકરો : -2.6771931648254395
પતિ : -0.2008996605873108
પૌત્ર : 0.3693620562553406
પુરુષ : 0.25360774993896484
વ્યક્તિ : 0.3871188759803772
જનતા : -1.136622667312622
માણસ : -0.12716799974441528
લોકો : -1.183879017829895
-1.3380554 -0.053220104 -0.51513773


In [ ]:
try:
    del model
except:
    pass
    
gc.collect()
torch.cuda.empty_cache()

model = AutoModelForMaskedLM.from_pretrained("google/muril-base-cased")

model.eval()
model.to(device)


model = unfreeze_partial(model, unfreeze="LN WPE WTE")


pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=GenderBiasDataset(train_professions),
    eval_dataset=GenderBiasDataset(test_professions),
    data_collator=data_collator,
    
)

trainer.train()

print("With LN WPE WTE debiasing".center(40, "#"))
female_scores, male_scores, neutral_scores  = evaluate_bias(test_professions)
print(female_scores, male_scores, neutral_scores)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--muril-base-cased/snapshots/afd9f36c7923d54e97903922ff1b260d091d202f/config.json
Model config BertConfig {
  "_name_or_path": "google/muril-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 197285
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--google--muril-base-cased/snapshots/afd9f36c7923d54e97903922ff1b260d091d202

Unfreezing LN
Unfreezing WPE
Unfreezing WTE
151948032


Epoch,Training Loss,Validation Loss
1,No log,3.514519
2,2.872600,3.185282
3,2.872600,2.904095


***** Running Evaluation *****
  Num examples = 630
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Configuration saved in ./results/checkpoint-500/generation_config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 630
  Batch size = 8
***** Running Evaluation *****
  Num examples = 630
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




#######With LN WPE WTE debiasing########
માતા : -1.0255253314971924
બહેન : -0.305290549993515
દીકરી : -2.5902397632598877
છોકરી : -4.2398905754089355
પત્ની : -2.3039770126342773
પૌત્રી : -0.3589768409729004
સ્ત્રી : -0.401379257440567
પિતા : 0.9623231291770935
ભાઈ : 1.1978331804275513
દીકરો : -1.1624573469161987
છોકરો : -2.404372453689575
પતિ : -0.40049391984939575
પૌત્ર : -0.3589768409729004
પુરુષ : 0.6128879189491272
વ્યક્તિ : 0.5818120241165161
જનતા : -0.36825060844421387
માણસ : 0.48086169362068176
લોકો : -0.37978509068489075
-1.6036112 -0.22189376 0.078659505


In [ ]:
# pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
"""
CONCLUSION
                    female     male       neutral
Without debiasing: -2.2249866 -0.7703856 -0.74644625

With LN debiasing: -1.0453371 0.09064193 -0.4234968
With LN WPE      : -1.3380554 -0.053220104 -0.51513773
debiasing
With LN WPE WTE  : -1.6036112 -0.22189376 0.078659505
debiasing

"""

In [2]:
def get_diff_percentage(orig, list_of_next):
    for i in list_of_next:
        print((abs(i) - abs(orig)) * 100 / abs(orig))

print("Feminine")
get_diff_percentage(-2.2249866,[
-1.0453371,
-1.3380554,
-1.6036112]
)

print("Masculine")
get_diff_percentage(-0.7703856 ,
[0.09064193,
-0.053220104,
 -0.22189376]
)

print("Neutral")
get_diff_percentage(-0.74644625,
[-0.4234968,
-0.51513773,
0.078659505]
)


Feminine
-53.018274357247805
-39.862316474175614
-27.927152460154137
Masculine
-88.23421284094614
-93.09175768602114
-71.19705248903927
Neutral
-43.264930328205146
-30.987967318477388
-89.46213407864265


In [ ]:
l = [-4.173, -2.575, -1.382]
sum=0
for i in l:
  j=i-l[1]
  sum=sum+(j*i)

std=sum/3  
std=std**0.5
mean=[]
for i in l:
  i=i-l[1]
  i=i/std
  mean.append(i)



In [ ]:
mean

[-1.2353707373206624, 0.0, 0.9222761512037237]

In [ ]:
print(std)

In [ ]:
print(mean)